# Drift Example

Simple 1 m drift.

In [ ]:
import impact.z as IZ
from impact.z.interfaces.bmad import export_particles as tao_export_particles
from pytao import Tao

from pmd_beamphysics import single_particle
from pmd_beamphysics.units import mec2

from impact.tests.z.conftest import bmad_files

In [ ]:
reference_particle_mass = mec2
energy = 10e6
gamma = energy / mec2
sigma_x0 = 0.001
norm_emit_x = 1e-12
norm_emit_y = 1e-12
beta_x = sigma_x0**2 * gamma / norm_emit_x
beta_y = beta_x

# Should give beta_x = 19569511.835591838 m
P0 = single_particle(pz=energy)

In [ ]:
!cat $bmad_files/drift.bmad

In [ ]:
tao = Tao(lattice_file=bmad_files / "drift.bmad", plot="mpl")

In [ ]:
tao.plot("beta", include_layout=False)

In [ ]:
P0.write("drift_initial_particles.h5")
tao.cmd("set beam_init position_file = drift_initial_particles.h5")

In [ ]:
tao.cmd(f"set beam_init n_particle = {len(P0)}")

In [ ]:
tao.cmd(f"set beam_init bunch_charge = {P0.charge}")

In [ ]:
tao.cmd("set beam_init saved_at = beginning d")

In [ ]:
tao.cmd("set global track_type = single")
tao.cmd("set global track_type = beam")

In [ ]:
tao.plot("beta", include_layout=False)

In [ ]:
# TODO write header showing repr as well
input = IZ.ImpactZInput.from_tao(tao)

In [ ]:
# Verify the ParticleGroup we dumped from Tao is the same one we specify as initial_particles
assert input.initial_particles == P0

In [ ]:
input.space_charge_off()
# input.space_charge_on(current=1.0)

In [ ]:
I = IZ.ImpactZ(input, use_temp_dir=False, workdir="./tmp", initial_particles=P0)
output = I.run(verbose=True)

In [ ]:
I.output

In [ ]:
Pin = I.output.particles["initial_particles"]
# Verify that ImpactZ read and wrote our initial particles correctly:
assert P0 == Pin

In [ ]:
P_raw = I.output.particles_raw["final_particles"]
P = I.output.particles["final_particles"]

In [ ]:
P.data

In [ ]:
Ptao = tao_export_particles(tao, "END")

In [ ]:
Ptao.data